## Abdomen grooming 

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange

from nmf_grooming import (
    NeuromechflyGrooming,
    all_groom_dofs, 
    plot_state_and_contacts,
    load_grooming_data,
)

from flygym.mujoco import Parameters, NeuroMechFly
from flygym.mujoco.arena import tethered

0 joint_Head
1 joint_Head_yaw
2 joint_Head_roll
3 joint_LHCoxa
4 joint_LHCoxa_roll
5 joint_LHCoxa_yaw
6 joint_LHFemur
7 joint_LHFemur_roll
8 joint_LHTibia
9 joint_LHTarsus1
10 joint_RHCoxa
11 joint_RHCoxa_roll
12 joint_RHCoxa_yaw
13 joint_RHFemur
14 joint_RHFemur_roll
15 joint_RHTibia
16 joint_RHTarsus1
17 joint_LPedicel
18 joint_LPedicel_yaw
19 joint_RPedicel
20 joint_RPedicel_yaw
21 joint_A1A2
22 joint_A3
23 joint_A4
24 joint_A5
25 joint_A6
26
3 10


Load grooming data

In [2]:
grooming_module_path = Path("./data/grooming_modules_provided_slow.pkl")
with open(grooming_module_path, "rb") as f:
    grooming_modules = pickle.load(f)
timestep = grooming_modules["timestep"]

## Simulation

In [3]:
sim_params = Parameters(
    timestep=timestep,
    render_mode="saved",
    render_playspeed=0.1,
    draw_contacts=True,
    render_camera="Animat/camera_front",
)

run_time = 1.0
target_num_steps = int(run_time / timestep) # 10'000
nmf = NeuromechflyGrooming(
    sim_params=sim_params,
)

obs_list = []
target_joint_angles_front = grooming_modules["foreleg"][:, :target_num_steps]
abd_arr = 0.25*np.ones((5, 558))
target_joint_angles_front = np.concatenate([target_joint_angles_front, abd_arr], axis=0)
print(np.shape(target_joint_angles_front))

nbre_step = len(grooming_modules['foreleg'][0])-1
# 10 loops of frontleg grooming
replay_steps = [nbre_step,2*nbre_step,3*nbre_step,4*nbre_step,5*nbre_step,
                6*nbre_step,7*nbre_step,8*nbre_step,9*nbre_step,10*nbre_step]
joint_pos = np.zeros(26)
k = 0
obs, info = nmf.reset()
for i in trange(10*558):
    joint_pos = target_joint_angles_front[:, k]
    
    action = {"joints": joint_pos}
    obs, reward, terminated, truncated, info = nmf.step(action)
    nmf.render()
    obs_list.append(obs)
    if i in replay_steps:
        k = 0
    else:
        k += 1
        
if sim_params.render_camera == "Animat/camera_front":
    frontleg_video = "frontleg_groom_front.mp4"
else:
    frontleg_video = "frontleg_groom_left.mp4"  
print(len(joint_pos), np.shape(joint_pos))  
nmf.save_video(frontleg_video)

(26, 558)


100%|██████████| 5580/5580 [00:11<00:00, 475.23it/s]


26 (26,)
